In [1]:
import pandas as pd
import numpy as np 
import re
import random
import copy

In [2]:
# The file that includes the slot and intent information.
df = pd.read_excel("Appliance Slot Intent.xlsx",encoding = "ISO-8859-1", sheet_name="Main V5.")
# The file that includes the types of values inside that slot. 
slot_df = pd.read_excel("Appliance Slot Intent.xlsx",encoding = "ISO-8859-1", sheet_name="Slot Description V4.")

In [3]:
df['TARGET_QUESTION']= df['TARGET_QUESTION'].str.lower()

In [4]:
df["TARGET_QUESTION"] = df['TARGET_QUESTION'].str.replace('[\.\?]','')

In [5]:
## Preprocessing slot_df dataframe

# Slot dataframe trimmed so that the Description is removed. 
slot_df_ = slot_df.iloc[:,[0,2]]
# replace the Example column with an array of the values separated by a comma
slot_df_ = slot_df_[slot_df_['Examples'].notnull()].copy()
slot_df_.loc[:,'Examples'] = slot_df_.loc[:,"Examples"].apply(lambda x: x.split(', '))
# dictionary of the slots. Key: slot value: possible vallues of the slot. 
slot_dict = dict(zip(slot_df_.Slot, slot_df_.Examples))

In [6]:
# Split the TARGET_QUESTION column and make it into an array of questions
df.iloc[:,1] = df.iloc[:,1].apply(lambda x: x.split('\n'))
#Temporary extract the TARGET_QUESTION column that was processed above. 
target = df.iloc[:,1]

In [7]:
target[]

SyntaxError: invalid syntax (<ipython-input-7-706449ec9ee0>, line 1)

In [8]:
# A Series of arrays representing the respective slots
slots = df.iloc[:,4].apply(lambda x: x.split(' '))
# The dictionary where key:Intent value:Question
target_dic = df.iloc[:,[0,1]]
# A dictionary of intents key: int value: intent
intent_dict = df.QUESTION_LABEL.to_dict()

In [9]:
slot_dict_boi = copy.deepcopy(slot_dict)
for key,value in slot_dict.items():
    list_slot = []
    for i in value:
        s = i.split()
        for j in range(len(s)):
            if j == 0:
                s[j] = s[j] + ":B-" + key[1:]
            else:
                s[j] = s[j] + ":I-" + key[1:]
        i = ' '.join(s)
        list_slot.append(i)
    slot_dict_boi[key] = list_slot

In [10]:
def boi_save(intent_number = len(target), single = False):
    f = open("data\\appliance\\train_test", "w")
    line = ""
    start = 0
    end = intent_number
    if (single == True):
        start = intent_number - 1
        end = intent_number
    for s in range(start,end): #Intent number
#         print("-----------------------------------------------")
#         print("Intent:",intent_dict[s],"|","number of slots:",len(slots[s]))
        for j in range(len(target[s])): #Sentence in the intent
            if (len(slots[s]) == 2):
                line,_ = boi_two_slots_write(s,j,f)    
            elif (len(slots[s]) == 3):
                line,_ = boi_three_slots_write(s,j,f)
            elif (len(slots[s]) == 1):
                line,_ = boi_one_slot_write(s,j,f)
#             print("-----------------------------------------------")
#     print("-----------------------------------------------")
    f.close()

def boi_one_slot_write(s,j,f):
    p = 0
    for i in range(len(slot_dict[slots[s][0]])): #Slot 1
            x = target[s][j]
            x = re.sub(slots[s][0], slot_dict_boi[slots[s][0]][i], x)
            x_boi = re.sub(r'((?:^|(?<= ))[a-zA-Z0-9]+(?= |$))', r'\1:O', x) 
            p += 1
            f.write(x_boi + " <=> " + intent_dict[s] + "\n")
            #print(x_boi)  
    return x, p

def boi_two_slots_write(s,j,f):
    p = 0
    x_boi = ""
    for i in range(len(slot_dict_boi[slots[s][0]])): #Slot 1
        for k in range(len(slot_dict_boi[slots[s][1]])):#Slot 2
            x_original = target[s][j]
            x = re.sub(slots[s][0], slot_dict_boi[slots[s][0]][i], x_original)
            x = re.sub(slots[s][1], slot_dict_boi[slots[s][1]][k], x)
            x_boi = re.sub(r'((?:^|(?<= ))[a-zA-Z0-9]+(?= |$))', r'\1:O', x) 
            p += 1
            f.write(x_boi+" <=> " + intent_dict[s] + "\n")
            #print(x_boi)

    return x, p

def boi_three_slots_write(s,j,f):
    p = 0
    for i in range(len(slot_dict[slots[s][0]])): #Slot 1
        for k in range(len(slot_dict[slots[s][1]])):#Slot 2
            for l in range(len(slot_dict[slots[s][2]])):#Slot 3
                x_original = target[s][j]
                x = re.sub(slots[s][0], slot_dict_boi[slots[s][0]][i], x_original)
                x = re.sub(slots[s][1], slot_dict_boi[slots[s][1]][k], x)
                x = re.sub(slots[s][2], slot_dict_boi[slots[s][2]][l], x)
                x_boi = re.sub(r'((?:^|(?<= ))[a-zA-Z0-9]+(?= |$))', r'\1:O', x) 
                p += 1
                f.write(x_boi+" <=> "+ intent_dict[s] + "\n")
                #print(x_boi)   
    return x, p

In [11]:
boi_save()

In [12]:
# Create appliance vocabulary for slots. 

slot_array = [] 
for key,value in slot_dict_boi.items():
    for i in value:
        slot_array = slot_array + i.split()
slot_array.sort()
slot_vocab = []
for i in slot_array:
    slot_vocab.append(i.split(':',1)[1])
    
slot_vocab.sort()
slot_vocab = list(dict.fromkeys(slot_vocab))

# for i in slot_vocab:
#     print(i)
    
f = open('data\\appliance\\vocab.slot','w')
for value in slot_vocab:
    f.write(value + "\n")
f.write("O")
f.close()

# Create appliance vocabulary for intents

f = open('data\\appliance\\vocab.intent','w')
for key,value in intent_dict.items():
    f.write(value + "\n")
f.close()

In [13]:
my_file = open('appliance\\train_test','r')
content = my_file.readlines()
random.shuffle(content)
print(len(content))
train = content[:int(len(content)*0.7)]
test = content[int(len(content)*0.7):int(len(content)*0.9)]
valid = content[int(len(content)*0.9):]
print(len(train))
print(len(test))
print(len(valid))

f = open('data\\appliance\\train','w')
for i in train:
    f.write(i)
    
f = open('data\\appliance\\test','w')
for i in test:
    f.write(i)
    
f = open('data\\appliance\\valid','w')
for i in test:
    f.write(i)

1361
952
272
137
